In [6]:
import re

# 3rd part imports
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from dashboard.logic.io import GSHEETS_URL, read_gsheet, comment_button, total_assets, \
    total_value_to_num, getDataFrames

In [8]:
# Read in summary DF and drop empty rows
df = read_gsheet(
    GSHEETS_URL, 
    header=None
)

# Extract sub DF to dictionary
df_dict = getDataFrames(df)

In [20]:
df_a = df_dict['announcements'].copy()
df_a = df_a.reset_index(drop=True)
df_a = df_a[0].str.split(pat=':', n=1, expand=True)
df_a.columns = ['date', 'text']
no_date_idx = df_a['date'].str.contains(r"property purchase 23rd", case=False).argmax()
df_a.loc[no_date_idx, 'text'] = df_a.loc[no_date_idx, 'date']
df_a.loc[no_date_idx, 'date'] = 'Jan 2023'
df_a['heading'] = ['Property Purchase', 'Rising Rates', 'Stocks Ralley and Crash', 
                   'No Change', 'Assets Falling', 'Cash is King', 'Stockbiling Cash', 
                   'Accumulating Cash', 'Into Cash!']
df_a

,date,text,heading
0,Jan 2023,Property Purchase 23rd Jan 2023. Cash reserve ...,Property Purchase
1,Jan 2023,As central banks tighten (QT + Interest rate ...,Rising Rates
2,Jan 2023,I wouldn't be surprised if we see a small ral...,Stocks Ralley and Crash
3,Dec 2022,No change to last month. 2022 financial forec...,No Change
4,Nov 2022,Asset prices are now beginning to fall in cor...,Assets Falling
5,Oct 2022,Cash is king right now. Assets are finally st...,Cash is King
6,Sep 2022,Cash currently held between multiple bank acc...,Stockbiling Cash
7,Aug 2022,"As per last month, I am still accumulating ca...",Accumulating Cash
8,Jul 2022,I'm following my February plan of going to CA...,Into Cash!
